In [13]:
#汎用的なTSKファジィ推論モデルを構築する
import numpy as np
import pandas as pd
import random
import math
import sys
import time

In [14]:
#各種データを読み込む
original_data = pd.read_csv('fuzzy_diabetes2_1.csv')

In [15]:
#次に各種データに対して正規化を行っていく
#ただし、例外のカラムを設けておく
#正規化をあらかじめ行いたいカラムについてはその例外にカラム名を付与しておく

#まず例外カラム名の制定
#各種データごとに確認・変更してほしい箇所
##########################################################################################################################
##########################################################################################################################
exception_columns = []
for clm in original_data.columns:
    if clm not in exception_columns:
        original_data[clm] = (original_data[clm] - original_data[clm].min()) / (original_data[clm].max() - original_data[clm].min())
##########################################################################################################################
##########################################################################################################################

In [16]:
#ここで各種データをnumpy配列に変換しておく
np_original_data = np.array(original_data)

In [17]:
#次に教師データとテストデータに分割する
#ここでは教師データ:テストデータを5:5とする

#教師データ、テストデータの割合の設定
#各種データごとに確認・変更してほしい箇所
##########################################################################################################################
##########################################################################################################################
TEACHER_SIZE_RATIO = 5
TEST_SIZE_RATIO = 5
##########################################################################################################################
##########################################################################################################################

#上で定めた割合に基づき、教師データ数とテストデータ数を求める
TEACHER_SIZE = ((int)(len(np_original_data) * ((TEACHER_SIZE_RATIO) / (TEACHER_SIZE_RATIO + TEST_SIZE_RATIO)))) + 1
TEST_SIZE = len(np_original_data) - TEACHER_SIZE

In [18]:
#次に教師データとテストデータに分割する
teacher_data = np_original_data[:TEACHER_SIZE,:]
test_data = np_original_data[TEACHER_SIZE:,:]

In [19]:
#元データの入力データと出力データの合計の属性数を求める
NUMBER_OF_DATA_VARIATION = len(teacher_data[0,:])

In [20]:
#説明変数と被説明変数とに分割する
#被説明変数は最後の要素のみとする
x_teacher_data = teacher_data[:,:NUMBER_OF_DATA_VARIATION-1]
y_teacher_data = teacher_data[:,NUMBER_OF_DATA_VARIATION-1]
x_test_data = test_data[:,:NUMBER_OF_DATA_VARIATION-1]
y_test_data = test_data[:,NUMBER_OF_DATA_VARIATION-1]

In [21]:
#次にファジィ分割数と入力データの属性数からファジィルールの総数を求める
#各種データごとに確認・変更してほしい箇所
##########################################################################################################################
##########################################################################################################################
NUMBER_OF_FUZZY_PARTITION = 3
NUMBER_OF_INPUT = NUMBER_OF_DATA_VARIATION - 1
NUMBER_OF_FUZZY_RULE = NUMBER_OF_FUZZY_PARTITION ** NUMBER_OF_INPUT
##########################################################################################################################
##########################################################################################################################

In [22]:
#前件部の初期値と後件部の初期値を設定する
#前件部の中心と幅は、ルール数×各ルールにおける入力変数の数
antecedent_center = np.empty((NUMBER_OF_FUZZY_RULE, NUMBER_OF_INPUT))
antecedent_broad = np.empty((NUMBER_OF_FUZZY_RULE, NUMBER_OF_INPUT))
consequent_part = np.empty((NUMBER_OF_FUZZY_RULE, TEACHER_SIZE+1))
consequent = np.zeros(NUMBER_OF_FUZZY_RULE)

In [23]:
#まず前件部の値を初期化する
#ただし、ここは簡略型ファジィ推論と同様
for i in range(NUMBER_OF_FUZZY_RULE):
    for j in range(NUMBER_OF_INPUT):
        if ((int)(i / (NUMBER_OF_FUZZY_PARTITION ** (NUMBER_OF_INPUT - (j+1))))) % NUMBER_OF_FUZZY_PARTITION == 0:
            antecedent_center[i,j] = 0
            antecedent_broad[i,j] = 1
        elif ((int)(i / (NUMBER_OF_FUZZY_PARTITION ** (NUMBER_OF_INPUT - (j+1))))) % NUMBER_OF_FUZZY_PARTITION == 1:
            antecedent_center[i,j] = 0.5
            antecedent_broad[i,j] = 0.5
        else:
            antecedent_center[i,j] = 1
            antecedent_broad[i,j] = 1

In [24]:
#次に後件部の値を初期化する
#後件部の値は簡略型とは違う
#教師データに使う73データと定数の74項から後件部実数値は構成される
#74番目の値は定数とする
for i in range(NUMBER_OF_FUZZY_RULE):
    for j in range(TEACHER_SIZE+1):
        consequent_part[i,j] = 0.01
for i in range(NUMBER_OF_FUZZY_RULE):
    for j in range(TEACHER_SIZE+1):
        if j == TEACHER_SIZE:
            consequent[i] = consequent[i] + consequent_part[i,j]
        else:
            consequent[i] = consequent[i] + (consequent_part[i,j] * y_teacher_data[j])
            #print(consequent_part[i,j] * y_teacher_data[j])
print(consequent)

[0.47 0.47 0.47 0.47 0.47 0.47 0.47 0.47 0.47 0.47 0.47 0.47 0.47 0.47
 0.47 0.47 0.47 0.47 0.47 0.47 0.47 0.47 0.47 0.47 0.47 0.47 0.47 0.47
 0.47 0.47 0.47 0.47 0.47 0.47 0.47 0.47 0.47 0.47 0.47 0.47 0.47 0.47
 0.47 0.47 0.47 0.47 0.47 0.47 0.47 0.47 0.47 0.47 0.47 0.47 0.47 0.47
 0.47 0.47 0.47 0.47 0.47 0.47 0.47 0.47 0.47 0.47 0.47 0.47 0.47 0.47
 0.47 0.47 0.47 0.47 0.47 0.47 0.47 0.47 0.47 0.47 0.47 0.47 0.47 0.47
 0.47 0.47 0.47 0.47 0.47 0.47 0.47 0.47 0.47 0.47 0.47 0.47 0.47 0.47
 0.47 0.47 0.47 0.47 0.47 0.47 0.47 0.47 0.47 0.47 0.47 0.47 0.47 0.47
 0.47 0.47 0.47 0.47 0.47 0.47 0.47 0.47 0.47 0.47 0.47 0.47 0.47 0.47
 0.47 0.47 0.47 0.47 0.47 0.47 0.47 0.47 0.47 0.47 0.47 0.47 0.47 0.47
 0.47 0.47 0.47 0.47 0.47 0.47 0.47 0.47 0.47 0.47 0.47 0.47 0.47 0.47
 0.47 0.47 0.47 0.47 0.47 0.47 0.47 0.47 0.47 0.47 0.47 0.47 0.47 0.47
 0.47 0.47 0.47 0.47 0.47 0.47 0.47 0.47 0.47 0.47 0.47 0.47 0.47 0.47
 0.47 0.47 0.47 0.47 0.47 0.47 0.47 0.47 0.47 0.47 0.47 0.47 0.47 0.47
 0.47 

In [ ]:
%%time

#次に教師データを用いて学習を行う
#ここでは前件部の更新はフェードアウトしておく
#学習回数を設定する
TRAIN_TIME = 1000

mean_squared_error = 0

#学習係数を設定する
#LEARNING_ANTECEDENT_BROAD = 0.005
#LEARNING_ANTECEDENT_CENTER = 0.005
LEARNING_CONSEQUENT = 0.001
#まず、各ルールに基づいて入力変数をファジィ化（メンバシップ関数に代入）する
membership_function = np.empty((NUMBER_OF_FUZZY_RULE, NUMBER_OF_INPUT))

for time in range(TRAIN_TIME):
    for i in range(TEACHER_SIZE):
        delta_broad = np.empty((NUMBER_OF_FUZZY_RULE, NUMBER_OF_INPUT))
        delta_center = np.empty((NUMBER_OF_FUZZY_RULE, NUMBER_OF_INPUT))
        #適合度を初期化する
        adaptability = np.ones(NUMBER_OF_FUZZY_RULE)
        for j in range(NUMBER_OF_FUZZY_RULE):
            for k in range(NUMBER_OF_INPUT):
                if(x_teacher_data[i,k] >= antecedent_center[j,k] - antecedent_broad[j,k]) and x_teacher_data[i,k] <= antecedent_center[j,k]:
                    membership_function[j,k] = (x_teacher_data[i,k] - (antecedent_center[j,k] - antecedent_broad[j,k])) / antecedent_broad[j,k]
                    #delta_broad[j,k] = (antecedent_center[j,k] - x_teacher_data[i,k]) / (antecedent_broad[j,k]**2)
                    #delta_center[j,k] = -(1 / antecedent_broad[j,k])
                elif(x_teacher_data[i,k] > antecedent_center[j,k]) and (x_teacher_data[i,k] <= antecedent_center[j,k] + antecedent_broad[j,k]):
                    membership_function[j,k] = -(x_teacher_data[i,k] - (antecedent_center[j,k] + antecedent_broad[j,k])) / antecedent_broad[j,k]
                    #delta_broad[j,k] = -(antecedent_center[j,k] - x_teacher_data[i,k]) / (antecedent_broad[j,k]**2)
                    #delta_center[j,k] = 1 / antecedent_broad[j,k]
            #各ルールにおける適合度を求める
            for k in range(NUMBER_OF_INPUT):
                adaptability[j] = adaptability[j] * membership_function[j,k]
        #各データに対して予測結果を求める
        output = 0
        output = np.sum(np.dot(adaptability, consequent)) / np.sum(adaptability)
        if(time == TRAIN_TIME-1):
            print(output, y_teacher_data[i])
            mean_squared_error = mean_squared_error + (output - y_teacher_data[i])**2
            if(i == TEACHER_SIZE-1):
                mean_squared_error = mean_squared_error / TEACHER_SIZE
                print("平均二条誤差")
                print(mean_squared_error)
        #出力値を求めた次に後件部と前件部の更新を行う
        #まずは後件部の値の更新を行う
        #consequent_partの更新を行った後に、consequentに変換を行う
        for j in range(NUMBER_OF_FUZZY_RULE):
            for k in range(TEACHER_SIZE + 1):
                if k == TEACHER_SIZE:
                    consequent_part[j,k] = consequent_part[j,k] + (LEARNING_CONSEQUENT * adaptability[j] / np.sum(adaptability)) * (y_teacher_data[i] - output)
                else:
                    consequent_part[j,k] = consequent_part[j,k] + (LEARNING_CONSEQUENT * adaptability[j] / np.sum(adaptability)) * (y_teacher_data[i] - output) * y_teacher_data[k]
        #consequentの更新を行う
        for j in range(NUMBER_OF_FUZZY_RULE):
            consequent[j] = 0
            for k in range(TEACHER_SIZE+1):
                if k == TEACHER_SIZE:
                    consequent[j] = consequent[j] + consequent_part[j,k]
                else:
                    consequent[j] = consequent[j] + (consequent_part[j,k] * y_teacher_data[k])
        #次に前件部の値の更新を行う
        #for j in range(NUMBER_OF_FUZZY_RULE):
            #for k in range(NUMBER_OF_INPUT):
                #antecedent_center[j,k] = antecedent_center[j,k] + (LEARNING_ANTECEDENT_CENTER * adaptability[j] / np.sum(adaptability)) * (y_teacher_data[i] - output) * (consequent[j] - output) * delta_center[j,k]
                #antecedent_broad[j,k] = antecedent_broad[j,k] + (LEARNING_ANTECEDENT_BROAD * adaptability[j] / np.sum(adaptability)) * (y_teacher_data[i] - output) * (consequent[j] - output) * delta_broad[j,k]

In [ ]:
#次にテストデータを用いて予測結果を比較する
#各ルールに対して全ての入力変数データをファジィ化する
test_membership_function = np.empty((NUMBER_OF_FUZZY_RULE, NUMBER_OF_INPUT))

test_mean_squared_error = 0

for i in range(TEST_SIZE):
    #適合度を初期化する
    test_adaptability = np.ones(NUMBER_OF_FUZZY_RULE)
    for j in range(NUMBER_OF_FUZZY_RULE):
        for k in range(NUMBER_OF_INPUT):
            if(x_test_data[i,k] >= antecedent_center[j,k] - antecedent_broad[j,k]) and x_test_data[i,k] <= antecedent_center[j,k]:
                test_membership_function[j,k] = (x_test_data[i,k] - ( antecedent_center[j,k] - antecedent_broad[j,k])) / antecedent_broad[j,k]
            elif(x_test_data[i,k] > antecedent_center[j,k]) and (x_test_data[i,k] <= antecedent_center[j,k] + antecedent_broad[j,k]):
                test_membership_function[j,k] = -(x_test_data[i,k] - (antecedent_center[j,k] + antecedent_broad[j,k])) / antecedent_broad[j,k]
        #各ルールにおける適合度を求める
        for k in range(NUMBER_OF_INPUT):
            test_adaptability[j] = test_adaptability[j] * test_membership_function[j,k]
    #各データに対して予測結果を求める
    test_output = 0
    test_output = np.sum(np.dot(test_adaptability, consequent)) / np.sum(test_adaptability)
    print(test_output, y_test_data[i])
    test_mean_squared_error = test_mean_squared_error + (test_output - y_test_data[i])**2
    if(i == TEST_SIZE-1):
        test_mean_squared_error = test_mean_squared_error / TEST_SIZE
        print("平均二条誤差")
        print(test_mean_squared_error)